In [ ]:
# print date and time of script execution
import datetime

# Enable interactive plots (%matplotlib -l to list backends)
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from neuron import h
h.load_file('stdrun.hoc')

# Our own modules
import bgcellmodels.mechanisms.synapses
from bgcellmodels.common import analysis

# GABA synapse model

In [ ]:
rng_seed = 8
highest_indices = {rng_seed: 0} # max high_index for each low_index (seed)
stim_data = {}
target_secs = []

rec_hobj = {}
trace_specs = {}


def make_gabab_synapses(num_synapses, num_spikes, rate, noise=1.0, bursty=False):
    """
    Create independent Sections with one synapse each to test effect of:
        - different stimulation rates
        - different synapse settigns
    """
    for i in range(num_synapses):

        # Make a container section
        label = str(i)
        secname = 'sec_' + label
        sec = h.Section(name=secname)
        target_secs.append(sec)
        target_seg = sec(0.5)
        rec_hobj[secname] = target_seg

        # HOWTO:
        # - to calibrate, stimulate at max rate for increasing burst lengths and see
        #   if conductance kinetics is as desired
        # - then test using random/noisy stimulation at close to mean physiological rate

        #######################################################################################
        # STIMULATION PATTERN

        # Stimulation pattern
        tstart = 50
        stim_rate = 50.0 # discharge rate in between pauses
        stim_interval = stim_rate**-1*1e3
        pause_dur = 300.0 # duration of pauses
        discharge_dur = 400.0 # duration of firing in between pauses
        dur_max_ms = 10000.0

        # RNG settings
        num_indep_repicks = dur_max_ms / stim_interval + 1000
        low_index = rng_seed
        highest_index = highest_indices.get(low_index, 0)
        high_index = int(highest_index + num_indep_repicks)
        highest_indices[low_index] = high_index # update highest index

        # MCellRan4: each stream should be statistically independent as long as 
        # the highindex values differ by more than the eventual length of the stream.
        # See http://www.neuron.yale.edu/neuron/static/py_doc/programming/math/random.html?highlight=MCellRan4
        stimrand = h.Random() # see CNS2014 Dura-Bernal example or EPFL cell synapses.hoc file
        stimrand.MCellRan4(high_index, low_index)
        stimrand.negexp(1) # if num arrivals is poisson distributed, ISIs are negexp-distributed

        # make bursting NetStim
        if bursty:
            stimsource = h.BurstStim()
            stimsource.fast_invl = stim_interval
            stimsource.slow_invl = pause_dur
            stimsource.burst_len = discharge_dur*1e-3*stim_rate
            stimsource.start = tstart
            stimsource.noise = noise
            stimsource.noiseFromRandom(stimrand) # Set it to use this random number generator
        else:
            # Make poisson spike generator
            stimsource = h.NetStim() # Create a NetStim
            stimsource.start = tstart
            stimsource.interval = stim_interval
            stimsource.number = num_spikes[i] # 4 * (i+1)
            stimsource.noise = noise
            stimsource.noiseFromRandom(stimrand) 
        rec_hobj['stim_' + label] = stimsource

        #######################################################################################
        # SYNAPSE PARAMETERS

        # GABA Synapse
        syn = h.GABAsyn2(target_seg)
        rec_hobj['syn_' + label] = syn

        nc = h.NetCon(stimsource, syn)
        nc.delay = 1.0
        nc.weight[0] = 1.0
        rec_hobj['spk_' + label] = nc

        # STP parameters
        syn.tau_rec = 400.0 # 100.0 * (i+1)
        syn.tau_facil = 1.0 # 100.0 * (i+1)
        syn.U1 = P_release = 0.2

        # GABA-A conductance
        # NOTE: set to weight to 1 and time constands same as GABA-B so that g_GABAB
        #       is the same as G-protein level in GABA-B cascade
        syn.gmax_GABAA = 1.0 # conductance in [nS]
        syn.tau_r_GABAA = 5.0
        syn.tau_d_GABAA = 25.0

        # GABA-B conductance
        syn.gmax_GABAB = 1.0
        syn.tau_r_GABAB = 5
        syn.tau_d_GABAB = 25
        
        # Signaling cascade
        syn.n = 4.0
        syn.KD = 1.4        # half-maximum of sigmoid(G)
                            # works well if set to between 2x - 3x peak value of thresholded signal

        syn.K3 = 0.098 # G-protein/LPF rise rate (default 0.098)
        syn.K4 = 1.0 / 160 # G-protein/LPF decay rate (default 0.033)
    #     syn.K1 = 10.0 * syn.K1
    #     syn.K2 = 10.0 * syn.K2
    #     syn.theta_R = -1.0 # <0 is no sigmoid
    #     syn.sigma_R = 0.2

        # Save inputs
        stim_data.setdefault('NetStims', []).append(stimsource)
        stim_data.setdefault('RNGs', []).append(stimrand)
        stim_data.setdefault('GABA_NetCons', []).append(nc)
        stim_data.setdefault('GABA_synapses', []).append(syn)

        #########################################################
        # Recording

        trace_specs.update({
            'AP_' + label:     {'netcon': 'spk_' + label},
            # Conductances and currents
            'gGABAA_' + label: {'pointp': 'syn_' + label, 'var':'g_GABAA'},
            'gGABAB_' + label: {'pointp': 'syn_' + label, 'var':'g_GABAB'},
            'iGABAA_' + label: {'pointp': 'syn_' + label, 'var':'i_GABAA'},
            'iGABAB_' + label: {'pointp': 'syn_' + label, 'var':'i_GABAB'},
            'itot_' + label:   {'pointp': 'syn_' + label, 'var':'i'},
            # Signaling cascade
            'Gprot_' + label:  {'pointp': 'syn_' + label, 'var':'G'},
    #         'R_' + label:  {'pointp': 'syn_' + label, 'var':'R'},
            # STP dynamics
            'Rrp_' + label:    {'pointp': 'syn_' + label, 'var':'Rrp'},
            'Use_' + label:    {'pointp': 'syn_' + label, 'var':'Use'},
        })


In [ ]:
num_synapses = 5
num_spikes = [1e9] * num_synapses
# num_spikes = [2, 5, 10, 25, 50]

make_gabab_synapses(num_synapses, num_spikes, rate=50.0, noise=1.0, bursty=False)

# Simulate & Analyze

In [ ]:
# Start recording
trace_specs.update({
    't_global': {'var': 't'},
})

rec_dt = 0.05
vec_dict, markers = analysis.recordTraces(rec_hobj, trace_specs, rec_dt)

In [ ]:
# Init and run simulation
h.dt = 0.025
h.celsius = 35.0
h.v_init = -68.0
h.tstop = 5000.0
h.init()

import time
tstart = time.time()
h.run()
tstop = time.time()
cputime = tstop - tstart
num_segments = sum((sec.nseg for sec in h.allsec()))
print("Simulated {} segments for {} ms in {} ms CPU time".format(
        num_segments, h.tstop, cputime))

## Plotting Options

In [ ]:
# Plotting options
interval = (4e3, 5e3)
page_width = 16
ax_height = 4

a = int(interval[0] // rec_dt)
b = int(interval[1] // rec_dt)
t_rec = vec_dict['t_global'].as_numpy()
t_plot = t_rec[a:b]

## Plot spike trains

In [ ]:
# Plot spikes
spike_data = analysis.match_traces(vec_dict, lambda label: label.startswith('AP'))
analysis.plotRaster(spike_data, interval, color='b', title='{} GLU spiketrains'.format(len(spike_data)))

## Plot activation functions

In [ ]:
# Plot G-protein dynamics and Hill activation function
syn = stim_data['GABA_synapses'][-1]


G = np.arange(0, 10, 0.001)
n = 4 # syn.n
Kd = 1.4 # syn.KD # 100.0
Gth = 0

hill_sigmoid = lambda G: G**n / (G**n + Kd**n)
hill_destexhe = lambda G: G**4 / (G**4 + 100.0)
activation = hill_sigmoid(G)
act_destexhe = hill_destexhe(G)
linear = np.arange(0, 10, 0.1)

fig, ax = plt.subplots()
ax.plot(G, activation, label='GABAsyn2')
ax.plot(G, act_destexhe, label='Destexhe')
ax.plot(linear, linear, label='y = x')

ax.set_ylim(-0.05, 1.05)
ax.set_xlim((-0.05, 2 * syn.KD)) # SETPARAM: set to range of G-protein concentration
ax.set_xlabel("G protein [mM]")
ax.set_ylabel("activation (0-1)")
ax.legend()
fig.suptitle('Thresholded G-protein concentration')

# Plot actual range of G during simulation
# fig, ax = plt.subplots()
# ax.plot(t_rec, vec_dict['G_str'].as_numpy())
# ax.set_title('STR : dynamics of G-protein variable in single syn')

# fig, ax = plt.subplots()
# ax.plot(t_rec, vec_dict['G_GPE'].as_numpy())
# ax.set_title('GPE : dynamics of G-protein variable in single syn')

In [ ]:
# fig, ax = plt.subplots()
# R_range = np.arange(0,3,0.01)
# theta_R = -1.0 # syn.theta_R
# sigma_R = 0.1 # syn.sigma_R
# logistic = lambda r: 1 / (1 + np.exp(-(r - theta_R) / sigma_R))
# ax.plot(R_range, logistic(R_range))
# ax.plot(theta_R, logistic(theta_R), 'r*')
# ax.plot(0.8, logistic(0.8), 'g*')

## Plot synapse state variables

In [ ]:
def plot_gaba_statevars(i):
    """
    Plot synaptic state variables
    """
    suffix = str(i)
    trace_prefixes = ['Rrp', 'Use', 'gGABAA', 'gGABAB'] # 'Gprot'
    
    Gprot = vec_dict['Gprot_'+suffix].as_numpy()
    
    fig, axes = plt.subplots(2, 1, figsize=(page_width, 1.2*ax_height))
    ax = axes[0]
    ax2 = ax.twinx()
    ax.plot(t_plot, vec_dict['Use_'+suffix].as_numpy()[a:b], 'g-', label='Use')
    ax.plot(t_plot, vec_dict['Rrp_'+suffix].as_numpy()[a:b], 'r-', label='Rrp')
    ax2.plot(t_plot, Gprot[a:b], 'b-', label='Gprot')
    ax2.hlines(syn.KD, t_plot[0], t_plot[-1], color='blue', linestyle=':', label='Kd')
    ax2.set_ylim(0.0, 1.1*Gprot.max())
#     ax.set_ylim((0, 1.1))
    ax.legend(loc='upper left')
    ax2.legend(loc='upper right')
    
    ax = axes[1]
    ax.plot(t_plot, vec_dict['gGABAA_'+suffix].as_numpy()[a:b], label='gGABA-A')
    ax.plot(t_plot, vec_dict['gGABAB_'+suffix].as_numpy()[a:b], label='gGABA-B')
#     ax.set_ylim((0, 1.1))
    ax.legend(loc='upper left')
    fig.suptitle('Synapse {} state variables'.format(suffix))
    
#     match_fun = lambda t: t.endswith(suffix) and any([t.startswith(prefix) for prefix in trace_prefixes])
#     n, KD = 4, 100.0 # parameters of kinetic scheme
#     hill_xform = lambda x: x**n/(x**n + KD)
#     syn_traces = analysis.match_traces(vec_dict, match_fun)
#     analysis.plotTraces(syn_traces, rec_dt, 
#                         traceSharex=True, title='Synaptic variables',
#                         traceXforms={'Gprot_' + str(i): hill_xform},
#                         timeRange=interval,
#                         figsize=(page_width, 1.2*ax_height))

In [ ]:
for i in range(num_synapses):
    plot_gaba_statevars(i)

## Plot synaptic currents

In [ ]:
# Plot synaptic variables
a = int(interval[0] // rec_dt)
b = int(interval[1] // rec_dt)
t_plot = t_rec[a:b]

# Plot GLU synapse variables
# fig, axes = plt.subplots(2, 1, figsize=(10,6))

# ax = axes[0]
# ax.plot(t_plot, vec_dict['gAMPA'].as_numpy()[a:b], 'b', label='gAMPA')
# ax.plot(t_plot, vec_dict['gNMDA'].as_numpy()[a:b], 'r', label='gNMDA')
# ax.set_ylabel('conductance (uS)')
# ax.legend()
# ax.grid(True)
# ax.set_title('Single GLU Synapse')

# ax = axes[1]
# ax.plot(t_plot, vec_dict['iAMPA'].as_numpy()[a:b], 'b', label='iAMPA')
# ax.plot(t_plot, vec_dict['iNMDA'].as_numpy()[a:b], 'r', label='iNMDA')
# ax.plot(t_plot, vec_dict['iGLU'].as_numpy()[a:b], 'g', label='i_tot')
# ax.set_ylabel('current (nA)')
# ax.legend()
# ax.grid(True)

# Plot GABA synapse variables
fig, axes = plt.subplots(2, 1, figsize=(10,6))

ax = axes[0]
ax.plot(t_plot, vec_dict['gGABAA'].as_numpy()[a:b], 'b', label='gGABAA')
ax.plot(t_plot, vec_dict['gGABAB'].as_numpy()[a:b], 'r', label='gGABAB')
ax.set_ylabel('conductance (uS)')
ax.legend()
ax.grid(True)
ax.set_title('Single GABA Synapse')

ax = axes[1]
ax.plot(t_plot, vec_dict['iGABAA'].as_numpy()[a:b], 'b', label='iGABAA')
ax.plot(t_plot, vec_dict['iGABAB'].as_numpy()[a:b], 'r', label='iGABAB')
ax.plot(t_plot, vec_dict['iGABA'].as_numpy()[a:b], 'g', label='i_tot')
ax.set_ylabel('current (nA)')
ax.grid(True)
ax.legend()

# Plot total GABA synapse current
itot_GABA = vec_dict['i_synGABA0'].as_numpy()
for i in range(1, len(stim_data['GABA_synapses'])):
    itot_GABA += vec_dict['i_synGABA{}'.format(i)].as_numpy()

itot_GLU = vec_dict['i_synGLU0'].as_numpy() * -1
for i in range(1, len(stim_data['GLU_synapses'])):
    itot_GLU -= vec_dict['i_synGLU{}'.format(i)].as_numpy()

plt.figure(figsize=(10,4))
plt.plot(t_rec, itot_GABA, 'r', label='i_GABA')
plt.plot(t_rec, itot_GLU, 'b', label='i_GLU')
plt.ylabel('current (nA)')
plt.xlabel('time (ms)')
plt.legend()
plt.grid()
plt.show()
plt.suptitle('Total GABA and GLU current')
# Plot remaining traces
# figs_vm = analysis.plotTraces(vec_dict, rec_dt, traceSharex=True) # yRange=(-80,40),